In [1]:
import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostRegressor
from concurrent.futures import ThreadPoolExecutor
from lightgbm import LGBMRegressor

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import numpy as np
from openfe import OpenFE, tree_to_formula, transform, TwoStageFeatureSelector, ForwardFeatureSelector
import pandas as pd
from pprint import pprint

import random

from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import time

from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)

experiment_name = 'openfe'

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [3]:
train.shape, test.shape

((1117957, 21), (745305, 20))

In [4]:
train.sample(3)

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
661336,5,3,4,5,8,8,7,3,4,2,7,6,2,7,3,6,9,7,6,5,0.540
922349,7,6,5,2,2,3,6,5,6,6,7,5,5,2,5,2,5,5,4,4,0.455
873763,5,0,5,4,5,3,3,7,10,7,2,4,3,7,6,4,8,6,4,8,0.540


In [5]:
feature_list = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors', 'FloodProbability']

In [6]:
TARGET = 'FloodProbability'

In [7]:
X = train.drop([TARGET], axis=1)
y = train[TARGET]

n_splits = 5
k5 = KFold(n_splits=n_splits, shuffle=True, random_state=5)

- Define Models list

In [8]:
# Define pipelines
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=50))
])
ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nystroem', Nystroem(n_components=500, random_state=5)),
    ('ridge', Ridge(alpha=0.1, max_iter=1000, random_state=5))
])

# Manually set pipeline names
knn_pipeline.name = 'KNN'
ridge_pipeline.name = 'Nystroem Ridge'

In [9]:
models = [
    CatBoostRegressor(random_state=5, verbose=False, early_stopping_rounds=100),
    ExtraTreesRegressor(random_state=5),
    HistGradientBoostingRegressor(random_state=5),
    LinearRegression(),
    LGBMRegressor(random_state=5, n_jobs=-1),
    RandomForestRegressor(random_state=5),
    knn_pipeline,
    ridge_pipeline,
    XGBRegressor(random_state=5),
]

- Create custom evaluation function

In [10]:
def evaluate_models(models, X, y, important_features, cv_split, experiment_name):
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train R2', 
                                        'MLA Test R2', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        if hasattr(alg, 'name'):
            MLA_name = alg.name
        else:
            MLA_name = alg.__class__.__name__

        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train R2': 0,
                'MLA Test R2': 0,
                'MLA Time': "0 min 0.00 sec",
            }

        X_train, X_test, y_train, y_test = train_test_split(X[features],
                                                            y,
                                                            test_size=0.1,
                                                            stratify=y,
                                                            shuffle=True,
                                                            random_state=5)

        start_time = time.time()
        alg.fit(X_train, y_train)
        end_time = time.time()

        # Evaluate the model
        train_score = r2_score(y_train, alg.predict(X_train))
        test_score = r2_score(y_test, alg.predict(X_test))

        # Results population
        result = {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train R2': train_score,
            'MLA Test R2': test_score,
            'MLA Time': f'{(end_time - start_time) / 60:.2f} min',
        }

        print(f'Done with {MLA_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            results_list.append(result)

    MLA_compare = pd.DataFrame(results_list)

    MLA_compare.sort_values(by=['MLA Test R2'], ascending=False, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [11]:
baseline_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    baseline_features[model_name] = list(X.columns)

In [12]:
%%time

baseline_models = evaluate_models(models, X, y, baseline_features, k5, f'{experiment_name}')
baseline_models

Done with LinearRegression.
Done with Nystroem Ridge.
Done with XGBRegressor.
Done with CatBoostRegressor.
Done with LGBMRegressor.
Done with HistGradientBoostingRegressor.
Done with KNN.
Done with ExtraTreesRegressor.
Done with RandomForestRegressor.
CPU times: total: 4h 33min 23s
Wall time: 1h 2min 56s


,MLA Name,MLA Parameters,MLA Train R2,MLA Test R2,MLA Time
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",0.849007,0.846446,12.74 min
3,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.844910,0.845309,0.07 min
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",0.817444,0.810189,8.38 min
7,Nystroem Ridge,"{'memory': None, 'steps': [('scaler', Standard...",0.792940,0.792933,3.51 min
4,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",0.772504,0.766484,14.07 min
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",0.772073,0.766351,32.48 min
6,KNN,"{'memory': None, 'steps': [('scaler', Standard...",0.719181,0.706257,0.03 min
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",1.000000,0.666396,53.57 min
5,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",0.951450,0.656895,60.29 min
